# Demo 1: Measurements and Likelihoods

In Demo 1, we will be focusing on different aspects of the generative model, as well as the first stage of inference (i.e., deriving a likelihood). 
The demo will be split into 3 parts, which we will explore sequentially throughout the first part of the lecture.
Each demo will be accompanied by **guiding questions**, which will help with the learning process as we go.
    
In **Part A**, we will explore properties of the *Measurement Distribution*.

In **Part B**, we will explore the relationship between the *Measurement Distribution* and the *Likelihood Distribution*.

Finally, In **Part C**, we will explore how differing patterns of *measurement noise* control the shape of the *Likelihood Distribution*.


In [63]:
# ---------------------------------------------------------------------------

## Demo 1A: The Measurement Distribution

In the current demo, we will explore the different properties of the *Measurement Distribution*. 
The below plot illustrates the measurement distribution in black and the stimulus value (s) as a dashed grey line. 
Two adjustable settings change the properties of the distribution.

1. "Stimulus" controls the location of the hand.
2. "Noise σ" controls the proprioceptive noise.

To make a measurement, click the box "Make Measurement". 
Doing so will randomly sample a sensory observation from the measurement distribution, and illustrate the observed value as a red dashed line.

    
### Guiding Questions
1. Keeping all other parameters at their default values, slide the "Stimulus" back and forth along the values. 
What changes about the distibution and what remains constant? 
What does this imply about the sensor (i.e., proprioceptors) and its ability to make measurements?

2. Now let's change the "Noise σ". If you increase or decrease its value, what happens to the Measurement Distribution?
                               
3. Finally, let's makes some observations with our measuring device. Choose a value for "Stimulus" and "Noise σ". 
Press the checkbox "Make Measurement" several times to make several measurements. 
How do these measurements relate to the actual stimulus location? What happens if you increase or decrease "Noise σ"? 


In [64]:
# ---------------------------------------------------------------------------
# Demo 1A

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, VBox, HBox, Button, Output
from IPython.display import display

# Make static images sharp inside notebooks
%matplotlib inline
plt.rcParams["figure.dpi"] = 120

# Global axis limits
S_RANGE = (-10, 10)
SIGMA_M_RANGE = (1, 5)
XLIM = (-20, 20)
YLIM = (-0.002, 0.05)

# Output widget for plot
output = Output()

# Controls
s_slider = FloatSlider(value=0.0, min=S_RANGE[0], max=S_RANGE[1], step=0.1,
                       description="Stimulus:", continuous_update=True,
                       readout_format=".1f")
sigma_slider = FloatSlider(value=2.0, min=SIGMA_M_RANGE[0], max=SIGMA_M_RANGE[1], step=0.1,
                           description="Noise σ:", continuous_update=True)

# Plotting function
def measurements(s: float, sigma_m: float, make_m: bool = False):
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.axvline(s, color="grey", linestyle="--")

    xs = np.linspace(XLIM[0], XLIM[1], 400)
    var_M = sigma_m**2
    s_norm = (1 / (np.sqrt(2 * np.pi) * var_M)) * np.exp(-0.5 * ((xs - 0)**2 / var_M)).sum()
    m_pdf = (1 / (np.sqrt(2 * np.pi) * var_M)) * np.exp(-0.5 * ((xs - s)**2 / var_M))
    m_pdf_scaled = m_pdf / s_norm
    ax.plot(xs, m_pdf_scaled, color="black", linewidth=2)

    if make_m:
        rng = np.random.default_rng()
        x_obs = rng.normal(loc=s, scale=sigma_m, size=1)
        ax.axvline(x_obs, color="red", linestyle="--")

    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    ax.set_xlabel("Hand Location")
    ax.set_ylabel("Probability")
    ax.text(0.45 * XLIM[1], 0.9 * YLIM[1], 'Measurement Dist.', color="black", fontsize=12)
    ax.text(0.45 * XLIM[1], 0.83 * YLIM[1], 'Measurement', color="red", fontsize=12)
    ax.grid(True, axis="x", linestyle=":", alpha=0.5)
    plt.show()

# Shared update function, called upon slider interaction or button press
def update_plot(make_m=False):
    with output:
        output.clear_output(wait=True)
        measurements(s_slider.value, sigma_slider.value, make_m=make_m)

# Button to trigger measurement
button = Button(description="Make Measurement", button_style='')

def on_button_click(b):
    update_plot(make_m=True)

button.on_click(on_button_click)

# Observe slider changes
s_slider.observe(lambda change: update_plot(), names='value')
sigma_slider.observe(lambda change: update_plot(), names='value')

# Initial display
ui_left = VBox([s_slider, sigma_slider, button])
display(HBox([ui_left, output]))
update_plot()
 
  

## Demo 1B: Effects of Measurement Noise on the Likelihood Distribution

In the current demo, we will explore the effects of measurement noise on the Likelihood Distribution.

As in the previous demo, "Stimulus" and "Noise σ" control the parameters of the Measurement Distribution, illustrated as a black curve
and a dashed grey line.
A new measurement can be taken by clicking the box "Make Measurement".

### Guiding Questions
1. Keeping all other parameters at their default values, press the box "Make Measurement" several times to make several measurements
How does the Likelihood Distribution relate to the Measurement Distribution?

2. Now let's change the "Noise σ". Increase or decrease the noise value. Now click "Make Measurement" (if it isn't clicked already).
What happens to the Likelihood Distribution? Take several measurements. What observations do you make about how the likelihood changes
from measurement to measurement? 
                               
3. Repeate the steps from question 2 across several different values of stimulus and noise. 
What consequence does measurement noise have on the Likelihood distributions?

In [65]:
# ---------------------------------------------------------------------------
# Demo 1B
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, Checkbox, VBox, HBox, Button, Output
from IPython.display import display

# Make static images sharp inside notebooks
%matplotlib inline
plt.rcParams["figure.dpi"] = 120

# Global axis limits
S_RANGE = (-10, 10)
SIGMA_M_RANGE = (1, 5)
XLIM = (-20, 20)
YLIM = (-0.002, 0.05)

# Output widget for plot
output2 = Output()

# Controls
s_slider2 = FloatSlider(value=0.0, min=S_RANGE[0], max=S_RANGE[1], step=0.1,
                       description="Stimulus:", continuous_update=True,
                       readout_format=".1f")
sigma_slider2 = FloatSlider(value=2.0, min=SIGMA_M_RANGE[0], max=SIGMA_M_RANGE[1], step=0.1,
                           description="Noise σ:", continuous_update=True)
# show_m_pdf_checkbox = Checkbox(value=True, description="Show Measurement curve")

# Store last measurement
last_measurement = {"x_obs": None}

# Plotting function
def measurements_likelihoods(s: float = 0.0, sigma_m: float = 1.0,
                              make_m2: bool = False):
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.axvline(s, color="grey", linestyle="--")

    xs = np.linspace(XLIM[0], XLIM[1], 400)
    var_M = sigma_m**2
    s_norm = (1 / (np.sqrt(2 * np.pi) * var_M)) * np.exp(-0.5 * ((xs - 0)**2 / var_M)).sum()

    # Measurement PDF (black)
    # if show_m_pdf:
    m_pdf = (1 / (np.sqrt(2 * np.pi) * var_M)) * np.exp(-0.5 * ((xs - s)**2 / var_M))
    m_pdf_scaled = m_pdf / s_norm
    ax.plot(xs, m_pdf_scaled, color="black", linewidth=2)

    # Make new measurement if requested
    if make_m2:
        rng = np.random.default_rng()
        x_obs = rng.normal(loc=s, scale=sigma_m, size=1)[0]
    #     last_measurement["x_obs"] = x_obs
    # else:
    #     x_obs = last_measurement["x_obs"]

    # Show likelihood if a measurement exists
    # if x_obs is not None:
        ax.axvline(x_obs, color="red", linestyle="--")
        var_L = sigma_m**2
        norm_l = (1 / (np.sqrt(2 * np.pi) * var_L)) * np.exp(-0.5 * ((0 - xs)**2 / var_L)).sum()
        pdf_L = (1 / (np.sqrt(2 * np.pi) * var_L)) * np.exp(-0.5 * ((x_obs - xs)**2 / var_L))
        pdf_L_norm = pdf_L / norm_l
        ax.plot(xs, pdf_L_norm, color="red", linewidth=2)

    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    ax.set_xlabel("Hand Location")
    ax.set_ylabel("Probability")
    ax.text(0.45 * XLIM[1], 0.9 * YLIM[1], 'Measurement Dist.', color="black", fontsize=12)
    ax.text(0.45 * XLIM[1], 0.83 * YLIM[1], 'Likelihood Dist.', color="red", fontsize=12)
    ax.grid(True, axis="x", linestyle=":", alpha=0.5)
    plt.show()

# Shared update function
def update_plot2(make_m2=False):
    with output2:
        output2.clear_output(wait=True)
        measurements_likelihoods(
            s_slider2.value,
            sigma_slider2.value,
            make_m2=make_m2
        )

# Button to trigger measurement
button2 = Button(description="Make Measurement", button_style='')

def on_button_click2(b):
    update_plot2(make_m2=True)

button2.on_click(on_button_click2)

# Observe slider and checkbox changes
s_slider2.observe(lambda change: update_plot2(), names='value')
sigma_slider2.observe(lambda change: update_plot2(), names='value')
# show_m_pdf_checkbox.observe(lambda change: update_plot(), names='value')

# Layout
ui_left2 = VBox([s_slider2, sigma_slider2, button2])
display(HBox([ui_left2, output2]))
update_plot2()
 
 

In [66]:
# ---------------------------------------------------------------------------

## Demo 1C: Illustrating a major distinction between Measurement and Likelihood Distributions

In the current demo, we will highlight the difference between measurement and likelihood, which is often hard to grasp at first.
In our opinion, this is best done by showing the effects of non-homegenous measurement noise on the shape of the likelihood.

We have added a new checkbox to this demo called "Gradient", which adds distance-dependent noise to the Measurement Distributions.
That is, the further from hand location at zero, the noisier the measurements are. The set value of "Gradient" determines the level of noiseiness.

### Guiding Questions
1. Keeping all other parameters at their default values, slide the "Stimulus" back and forth along the values. 
How does this relate to Demo 1A?

2. Let's add a "gradient" to the measurement noise. Change the value of "Gradient" to a moderate level (between 0.1 to 0.2).
Repeat the process from Question 1. How do the stimulus value and measurement noise relate?
What does this imply about the sensor (i.e., proprioceptors) and its ability to make measurements?

3. Now let's observe the likelihood curve. Click "Show likelihood curve". 
How does its shape change as you slide the "Stimulus" back and forth along the values?
Why does it take on this shape (hint: think about how the likelihood at value s_hyp relates to the measurement noise). 
                               
4. Repeat the above steps with different values for "Gradient". How does the shape of the Likelihood change? 

In [67]:
# ---------------------------------------------------------------------------
# Demo 1C

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, Checkbox, VBox, HBox, Button, Output, interactive
from IPython.display import display

GRAD_RANGE = (0, 0.5)

def m_l_inhomogeneous(s: float = 0.0, 
                     sigma_m: float = 1.0,
                     grad: float = 1.0,
                     show_m_pdf: bool = True,
                     show_L_pdf: bool = True):
    

    fig, ax = plt.subplots(figsize=(8, 6))


    # Line showing the true simulus
    ax.axvline(s, color="black", linestyle="--")


    # MEASUREMEMNT DISTRIBUTION
    xs = np.linspace(XLIM[0], XLIM[1], 400) # centers of the measurements

    sigma_grad = sigma_m + abs(s)*grad
    var_M = sigma_grad**2 # variance of the measurement
    s_norm = (1 / (np.sqrt(2 * np.pi) * var_M)) * np.exp(-0.5 * ((xs - 0)** 2 / var_M)).sum() # normalization constant for measurement curve
    
    # Measurement PDF
    if show_m_pdf:
        m_pdf = (1 / (np.sqrt(2 * np.pi) * var_M)) * np.exp(-0.5 * ((xs - s)** 2 / var_M))
        m_pdf_norm = m_pdf / s_norm # Scaling the pdf for better visual
        ax.plot(xs, m_pdf_norm, color="black", linewidth=2)
        

    # Show the inference (likelihood)
    if show_L_pdf:

        norm_l = []
        pdf_L = []
        for i in range(len(xs)):
            curr_x = xs[i]

            sigma_l_grad = sigma_m + abs(xs[i])*grad
            
            var_L = sigma_l_grad**2
            norm_l_temp = (1 / (np.sqrt(2 * np.pi) * var_L)) * np.exp(-0.5 * ((0-xs[i])** 2 / var_L)) # Normalization for likelihood
            pdf_L_temp = (1 / (np.sqrt(2 * np.pi) * var_L)) * np.exp(-0.5 * ((s-xs[i])** 2 / var_L)) # unnormalized likelihood
            norm_l.append(norm_l_temp)
            pdf_L.append(pdf_L_temp)
            
        pdf_L_norm = pdf_L / np.array(norm_l).sum() # Normalize the pdf
        ax.plot(xs, pdf_L_norm, color="red", linewidth=2)



    # Fixed axes
    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    ax.set_xlabel("Hand Location")
    ax.set_ylabel("Probability")
    ax.text(0.45*XLIM[1],0.9*YLIM[1],'Measurement Dist.',color="black",fontsize=12)
    ax.text(0.45*XLIM[1],0.83*YLIM[1],'Likelihood Dist.',color="red",fontsize=12)
    
    # Aesthetics
    ax.grid(True, axis="x", linestyle=":", alpha=0.5)
    plt.show()


# ---------------------------------------------------------------------------
# controls
controls = {
    "s": FloatSlider(value=0.0, min=S_RANGE[0], max=S_RANGE[1], step=0.1,
                       description="Stimulus:", continuous_update=True,
                       readout_format=".1f"),
    "sigma_m": FloatSlider(value=2.0, min=SIGMA_M_RANGE[0], max=SIGMA_M_RANGE[1], step=0.1,
                          description="Noise σ:", continuous_update=True),
    "grad": FloatSlider(value=0, min=GRAD_RANGE[0], max=GRAD_RANGE[1], step=0.05,
                          description="Gradient:", continuous_update=True),
    "show_m_pdf": Checkbox(value=True, description="Show Measurement curve"),
    "show_L_pdf": Checkbox(value=False, description="Show Likelihood curve"),

}

interactive_plot = interactive(m_l_inhomogeneous, **controls)

# Two‑column layout: sliders left, plot right
ui_left = VBox([controls["s"], controls["sigma_m"],controls["grad"],controls["show_m_pdf"],controls["show_L_pdf"]])

output_area = interactive_plot.children[-1]

display(HBox([ui_left, output_area]))
interactive_plot.update()
